In [13]:
from bs4 import BeautifulSoup
import spacy
from geopy.geocoders import OpenCage
from geopy.geocoders import Nominatim
from elasticsearch import Elasticsearch
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,'scheme': 'http'}],http_auth=('farah', 'farah123'))

index_name = "reuters_index_irproj"

C:\Users\Rahhal AbuZahra\AppData\Local\Temp\ipykernel_4460\3329905203.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port': 9200,'scheme': 'http'}],http_auth=('farah', 'farah123'))


In [14]:
nlp = spacy.load("en_core_web_sm")
opencage_api_key2 = '662f322da9c7453f8784cc641f08b0b7'
opencage_api_key = 'dc8ce9c87d2849aeb51e5e4402086b6c'
# geolocator = OpenCage(opencage_api_key)
geolocator = Nominatim(user_agent="geo-points-places")
geolocator2 = Nominatim(user_agent="geo-points-dateline")
# geolocator2 = OpenCage(opencage_api_key2)

In [4]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

mapping = {
    
 "settings": {
            "analysis": {
                "analyzer": {
                    "autocomplete_analyzer": {
                        "tokenizer": "autocomplete_tokenizer",
                        "filter": ["lowercase"]
                    }
                },
                "tokenizer": {
                    "autocomplete_tokenizer": {
                        "type": "edge_ngram",
                        "min_gram": 2,
                        "max_gram": 10,
                        "token_chars": ["letter", "digit"]
                    }
                }
            }
        },
    "mappings": {
        "properties": {
            "title": {
                    "type": "text",
                    "fields": {
                        "autocomplete": {
                            "type": "text",
                            "analyzer": "autocomplete_analyzer"
                        }
                    }
                },
            "body": {"type": "text"},
            "topic": {"type": "text"},
            "authors": {
                "type": "nested",
                "properties": {
                    "first_name": {"type": "text"},
                    "last_name": {"type": "text"},
                    "email": {"type": "text"}
                }
                },
            "date": {
              "type": "text",
              "fields": {
              "keyword": {
              "type": "keyword",
              "ignore_above": 256
                }
                }
                },
            "geopoint": {
                "type": "nested",
                "properties":{
                  "loc":{"type":"text"},
                  "coordinates":{"type":"geo_point"}
                }
                },
            "georeferences": {
                "type": "nested",
                "properties": {
                    "loc":{"type":"keyword"},
                }
                },
            "temporalExpressions": {
                "type": "nested",
                "properties": {
                    "expression": { "type": "text"},
                    "type": { "type": "keyword" },
               }
               }
    }
    }
}
# Create the index with the specified mapping and settings
response = es.indices.create(index=index_name, body=mapping, ignore=400)
print(response)

C:\Users\Rahhal AbuZahra\AppData\Local\Temp\ipykernel_4460\960257515.py:78: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.create(index=index_name, body=mapping, ignore=400)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'reuters_index_irproj'}


In [5]:
def read_sgm_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    soup = BeautifulSoup(content, 'html.parser')
    return soup

In [6]:
file_path ="D:/IRproject/docs/reut2-007.sgm"
soup = read_sgm_file(file_path)

In [7]:
nltk.download('punkt')
nltk.download('stopwords')

def process_text(content):
    words = word_tokenize(content)
    # Remove stop words and tokens with length less than 3
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words and len(word) >= 3]
    
    # Perform stemming
    ps = PorterStemmer()
    stemmed_words = [ps.stem(word) for word in filtered_words]
    
    # Join the processed words back into a string
    processed_text = ' '.join(stemmed_words)
    
    return processed_text

[nltk_data] Downloading package punkt to C:\Users\Rahhal
[nltk_data]     AbuZahra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rahhal
[nltk_data]     AbuZahra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from dateutil import parser

def format_date(date_string, output_format='%Y-%m-%d %H:%M:%S'):
    parsed_date = parser.parse(date_string)
    formatted_date = parsed_date.strftime(output_format)
    return formatted_date

In [15]:
MAX_RETRIES = 3
RETRY_DELAY_SECONDS = 2

def geocode_with_retry(location):
    retries = 0
    while retries < MAX_RETRIES:       
        try:
            coordinates = geolocator2.geocode(location)
            if coordinates:
                return {"loc": location,"lat": coordinates.latitude, "lon": coordinates.longitude}
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Geocoding request failed. Retrying... ({retries + 1}/{MAX_RETRIES})")
            retries += 1
            time.sleep(RETRY_DELAY_SECONDS)  # Add a delay between retries

    print(f"Max retries reached. Could not geocode location: {location}")
    return None

In [10]:
from dateutil import parser
import re

def classify_temporal_expression(expression):
    try:
        parsed_date = parser.parse(expression, fuzzy_with_tokens=True)
        if len(parsed_date) == 1:
            return "Fully-Qualified"
    except ValueError:
        pass

    deictic_keywords = ['now', 'today', 'tomorrow', 'yesterday', 'soon','today', 'yesterday', 'last', 'ago']
    if any(keyword in expression.lower() for keyword in deictic_keywords):
        return "Deictic"
    
    month_names = [
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ]

    if expression in month_names:
        return "Deictic"
    
    anaphoric_keywords = ['after', 'before', 'since', 'ago', 'when','that', 'same', 'previous','earlier','preceding']
    if any(keyword in expression.lower() for keyword in anaphoric_keywords):
        return "Anaphoric"
    
    fully_qualified_pattern = r'\bon\b|\bat\b|\bon\s\w+\s\d+|\bat\s\d+:\d+\s(?:AM|PM)|\bended\b'
    if re.search(fully_qualified_pattern, expression, re.IGNORECASE):
        return "Fully-Qualified"
    
    return "Unknown"


expression1 = "the 26 weeks ended December 28"
expression2 = "a year earlier"
expression3 = "the preceding six months"
expression4='February'

print(classify_temporal_expression(expression1))  
print(classify_temporal_expression(expression2))  
print(classify_temporal_expression(expression3))   
print(classify_temporal_expression(expression4))  



Fully-Qualified
Anaphoric
Anaphoric
Deictic


In [16]:
# Extracting information
counter=0
for reuters_tag in soup.find_all('reuters'):
    
    date = reuters_tag.find('date').text
    
    nested_author =[] 
    author_tag = reuters_tag.find('author')
    if author_tag:
        author_info = author_tag.text.strip()
        author_parts = author_info.split(' ')
        if len(author_parts) >= 3:
            first_name = author_parts[1]
            last_name = author_parts[2].replace(',', '')  
            print("First Name:", first_name)
            print("Last Name:", last_name)
            nested_author=[{'First Name':first_name,'Last Name':last_name}]
            
    title_tag = reuters_tag.find('title')
    title = title_tag.text if title_tag else "No title available"
    
    places_tag = reuters_tag.find('places')
    places_text = places_tag.find('d').text.strip() if places_tag and places_tag.find('d') else None

    dateline_tag = reuters_tag.find('dateline')
    dateline = dateline_tag.text if dateline_tag else None
    location_part = dateline.split()[0].replace(',', '') if dateline else None


    topic_tag = reuters_tag.find('topic')
    topic = topic_tag.find('d').text.strip() if topic_tag and topic_tag.find('d') else None

    body_tag = reuters_tag.find('body')
    body = body_tag.text if body_tag else "No body text available"

    # Process the text with spaCy
    doc = nlp(body)

    geopoints = []
    if places_text:
        coordinates = geolocator.geocode(places_text)
        if coordinates:
            geopoints.append({"loc": places_text,"lat": coordinates.latitude, "lon": coordinates.longitude})

    if location_part:
        loc_coordinates = geocode_with_retry(location_part)
        if loc_coordinates:
            geopoints.append(loc_coordinates)
    # If place or dateline tags is empty, extract from body
    if not geopoints:
        for ent in doc.ents:
            if ent.label_ == 'GPE':  
                location = ent.text
                coordinates = geolocator.geocode(location)
                if coordinates and not any(point['lat'] == coordinates.latitude and point['lon'] == coordinates.longitude for point in geopoints):
                    geopoints.append({"loc": location,"lat": coordinates.latitude, "lon": coordinates.longitude})
            
    
    # Extract georeferences
    georeferences = [{"loc":ent.text} for ent in doc.ents if ent.label_ == 'GPE']

    # Extract temporal expressions
    temporal_expressions = [
    {"expression": ent.text, "type": classify_temporal_expression(ent.text)} 
    for ent in doc.ents 
    if ent.label_ in ['DATE', 'TIME']
    ]


    # Index data into Elasticsearch
    document = {
        'date': format_date(date),
        'topic':topic,
        'title': title,
        'body':process_text(body),
        'authors':nested_author,
        'geopoints': geopoints,
        'georeferences':georeferences,
        'temporal_expressions':temporal_expressions
    }

    es.index(index=index_name, body=document)
    counter=counter+1
    print(counter)

# Refresh the index to make the changes visible to search
es.indices.refresh(index=index_name)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
First Name: Jorge
Last Name: Aguirre
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
First Name: Franz-Josef
Last Name: Ebel
51
52
53
54
55
56
57
58
59
60
61
First Name: Greg
Last Name: McCune
62
63
64
65
66
67
68
69
70
First Name: Lisa
Last Name: Vaughan
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
First Name: Donald
Last Name: Nordberg
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
First Name: Colin
Last Name: Brinsden
103
104
First Name: Paul
Last Name: Bolding
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
First Name: Claire
Last Name: Miller
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
First Name: Gerrard
Last Name: Raven
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
1

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})